In [2]:
import pandas as pd
import os

In [36]:
data = pd.read_csv("individuals.csv")

In [4]:
print(data)
hashmap = {}
for name in data["name"]:
    

                       film               name   result  gender  birthday
0            In Old Arizona      Warner Baxter   Winner    Male      1889
1               Thunderbolt    George Bancroft  Nominee    Male      1882
2                     Alibi     Chester Morris  Nominee    Male      1901
3               The Valiant          Paul Muni  Nominee    Male      1895
4               The Patriot        Lewis Stone  Nominee    Male      1879
...                     ...                ...      ...     ...       ...
1367              Nomadland         Chloé Zhao   Winner  Female      1982
1368          Another Round  Thomas Vinterberg  Nominee    Male      1969
1369                   Mank      David Fincher  Nominee    Male      1962
1370                 Minari    Lee Isaac Chung  Nominee    Male      1978
1371  Promising Young Woman    Emerald Fennell  Nominee  Female      1985

[1372 rows x 5 columns]


In [37]:
hashmap = {}
for names in data["name"]:
    if hashmap.get(names) == None:
        hashmap[names] = 1
    else:
        hashmap[names] = hashmap[names] + 1

number_of_nomination = []

for names in data["name"]:
   number_of_nomination.append(hashmap.get(names))

data["Number of nominations"] = number_of_nomination

In [38]:
data

,film,name,result,gender,birthday,Number of nominations
0,In Old Arizona,Warner Baxter,Winner,Male,1889,1
1,Thunderbolt,George Bancroft,Nominee,Male,1882,1
2,Alibi,Chester Morris,Nominee,Male,1901,1
3,The Valiant,Paul Muni,Nominee,Male,1895,6
4,The Patriot,Lewis Stone,Nominee,Male,1879,1
...,...,...,...,...,...,...
1367,Nomadland,Chloé Zhao,Winner,Female,1982,1
1368,Another Round,Thomas Vinterberg,Nominee,Male,1969,1
1369,Mank,David Fincher,Nominee,Male,1962,3
1370,Minari,Lee Isaac Chung,Nominee,Male,1978,1


In [29]:

print(max(data["Number of nomination"]))

17.0
